### Clone repository and install libraries
Note that due to package reinstalls, the session will have to be restarted

In [ ]:
!git clone https://github.com/vDawgg/awt-pj-ss24-finding_scenes-2
%cd awt-pj-ss24-finding_scenes-2/
!git checkout feature/llm-pipeline
!pip install -r requirements.txt

In [ ]:
!git clone https://github.com/vDawgg/awt-pj-ss24-finding_scenes-2
%cd awt-pj-ss24-finding_scenes-2/
!git checkout feature/extract-keyframes
!pip install -r requirements.txt

#### Write the token to the constants file so we can access gated repos
Run this cell after the session restarts
→ Add your HF access token with write permissions here

In [ ]:
# Make sure were in the right working dir

%cd /content/awt-pj-ss24-finding_scenes-2/
! sed -i "s/HF_TOKEN = \"\"/HF_TOKEN = \"YOUR_TOKEN_GOES_HERE\"/" utils/constants.py

### Download YouTube video

In [ ]:
from utils.video.video_extraction_with_pytube import YouTubeVideo

input_video = "https://www.youtube.com/watch?v=xRMPKQweySE"

downloader = YouTubeVideo(input_video)
path, subtitles = downloader.download_video_and_subtitles()

### Extract scenes from the video

In [ ]:
from utils.video.scenes import get_scenes

scene_csv = get_scenes(path)

### Extract keyframes from the video

In [2]:
from utils.keyframe.keyframe_extraction import generate_keyframes_from_scenes

video_name = "Rust in 100 Seconds"

keyframe_csv_path = generate_keyframes_from_scenes(
    video_name=video_name,
    no_of_frames_to_return=1
)

Completed processing for :  /home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/utils/../videos/video_scenes/Rust in 100 Seconds-Scene-001.mp4
Completed processing for :  /home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/utils/../videos/video_scenes/Rust in 100 Seconds-Scene-002.mp4
Completed processing for :  /home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/utils/../videos/video_scenes/Rust in 100 Seconds-Scene-003.mp4
Completed processing for :  /home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/utils/../videos/video_scenes/Rust in 100 Seconds-Scene-004.mp4
Completed processing for :  /home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/utils/../videos/video_scenes/Rust in 100 Seconds-Scene-005.mp4
Completed processing for :  /home/limin/Documents/programmin

In [3]:
print(keyframe_csv_path)

/home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/utils/../videos/Rust in 100 Seconds_keyframes/extracted_keyframes.csv


### Save subtitles to the keyframes csv

In [4]:
from utils.video.subtitles import save_subtitle_in_csv

# save_subtitle_in_csv(subtitles, "videos/keyframes/extracted_keyframes.csv")
save_subtitle_in_csv(subtitles, keyframe_csv_path)

NameError: name 'subtitles' is not defined

### Set up VLM

In [1]:
from utils.captioning.model import CaptionModel

model_id = "vikhyatk/moondream2"
revision = "2024-05-20"
model = CaptionModel(model_id, revision=revision)
directory = "./videos/keyframes"

/home/limin/anaconda3/envs/awt-pj-new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

### Generate captions for keyframes

In [ ]:
import os
import pandas as pd
from pathlib import Path
from utils.captioning.caption_keyframes import caption_images

csv_filepath = "extracted_keyframes.csv"

filename_column = 'Filename'
directory = os.path.join("/content","keyframes")
# directory = "/home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/videos/Rust in 100 Seconds_keyframes"
csv_filepath = Path(directory) / csv_filepath
csv_df = pd.read_csv(csv_filepath)

tasks = {
    "CAPTION": "Caption the scene. Describe it with as much information as possible.",
    "INFORMATION": "Generate detailed information for this scene.",
    "LANGUAGE": "What is the language used in the video this keyframe was captured from?",
    "VIDEO_TYPE": "What kind of video is this, is it a tutorial, a lecture, etc?",
}

prompt = f"""
Given keyframe extracted from a scene and the corresponding SUBTITLES - the subtitles transcribed for this scene.
Generate detailed information for this scene for TASK - instructions on what exactly to capture.
Use both the image and the subtitles to infer the information.
If the TASK cannot be completed, then return "NONE".
""".strip()

# csv_df = caption_images(model=model, base_prompt=prompt, tasks=tasks, directory=directory)
csv_df = caption_images(
    model=model,
    base_prompt=prompt,
    tasks=tasks,
    directory=directory,
    csv_df=csv_df
)
csv_df.to_csv(csv_filepath, index=False)


In [ ]:
    csv_filepath = "extracted_keyframes.csv"

    filename_column = 'Filename'
    directory = os.path.join("/content","keyframes")
    directory = "/home/limin/Documents/programming/finding_scenes_in_learning_videos/awt-pj-ss24-finding_scenes-2/videos/Rust in 100 Seconds_keyframes"
    csv_filepath = Path(directory) / csv_filepath
    csv_df = pd.read_csv(csv_filepath)

    model_id = "vikhyatk/moondream2"
    revision = "2024-05-20"
    model = CaptionModel(model_id, revision=revision)

    tasks = {
        "CAPTION": "Caption the scene. Describe it with as much information as possible.",
        "INFORMATION": "Generate detailed information for this scene for this scene.",
        "LANGUAGE": "What is the language used in the video this keyframe was captured from?",
        "VIDEO_TYPE": "What kind of video is this, is it a tutorial, a lecture, and the likes.",
    }

    prompt = f"""
    Given keyframe extracted from a scene and the corresponding SUBTITLES - the subtitles transcribed for this scene.
    Generate detailed information for this scene for TASK - instructions on what exactly to capture.
    Use both the image and the SUBTITLES to infer the information.
    If SUBTITLES is not provided, infer the information only from the keyframe image.
    If the TASK cannot be completed, then return "NONE".
    """.strip()

    csv_df = caption_images(
        model=model,
        base_prompt=prompt,
        tasks=tasks,
        directory=directory,
        csv_df=csv_df
    )

    csv_df.to_csv(csv_filepath, index=False)


### Set up VLM

In [ ]:
from utils.llm.model import LLMModel

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = LLMModel(model_id, load_in_4bit=True)

### Contextualize the keyframe captions using the LLM

In [ ]:
from utils.llm.align_captions import align_video_captions

align_video_captions(model, "./videos/keyframes/extracted_keyframes.csv", tasks)

### Creating Metadataobject


In [ ]:
from utils.metadata.metadata_function import get_metadata_from_scene_file, get_metadata_from_keyframe_file
from utils.objects.metadata_object import MetaDataObject

scene_objects_with_extraction_data=get_metadata_from_scene_file(path_to_scene_csv=scene_csv)
scene_objects_with_llm_data=get_metadata_from_keyframe_file( path_to_keyframes_csv="./videos/keyframes/extracted_keyframes.csv" ,scene_objects= scene_objects_with_extraction_data,tasks=tasks)
metaDataObject=MetaDataObject(input_video, downloader.yt, scene_objects_with_llm_data) 
print(metaDataObject.to_json())